In [1]:
import sys
sys.path.insert(0, '..//src')
sys.path.insert(0, '..//src//data')
sys.path.insert(0, '..//src//features')
sys.path.insert(0, '..//src//models')

import global_func as gf
import data_preprocessing
import predict
import fit_model

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

# Load Clean Data

In [2]:
DataPrep = data_preprocessing.DataPreprocessing()
df_train = gf.load_pkl(pkl=r'..\data\processed\df_train_05.pkl')
df_valid = gf.load_pkl(pkl=r'..\data\processed\df_valid_05.pkl')
df_test = gf.load_pkl(pkl=r'..\data\processed\df_test_05.pkl')

df_train.name = 'df_train'
df_valid.name = 'df_valid'
df_test.name = 'df_test'

#gf.plotting_line_all(dataframe=df_train)

# Modelling

## Fitting Model

In [ ]:
model = fit_model.modelling(train_dataframe=df_train)

## Model Summary

In [ ]:
print(model.summary())

## Save to PKL

In [ ]:
#gf.create_pkl(obj=model, pkl=r'..\models\pkl\sarimax(0,0,0)(2,1,0)(12).pkl')

# Predicting

In [4]:
#sarimax = gf.load_pkl(pkl=r'..\models\pkl\sarimax(0,0,0)(2,1,0)(12).pkl')
#scaler = gf.load_pkl(pkl=r'..\models\pkl\scaler.pkl')
#Predictor = predict.Predictor(model=sarimax, scaler=scaler)
#gf.create_pkl(obj=Predictor, pkl=r'..\models\pkl\predictor.pkl')
#Predictor = gf.load_pkl(pkl=r'..\models\pkl\predictor.pkl')
Predictor = gf.load_pkl(pkl=r'..\.cannot_upload\predictor.pkl')

df_forecast_valid = Predictor.forecasting(dataframe=df_valid, name='pred_valid')
df_forecast_test = Predictor.forecasting(dataframe=df_test, name='pred_test')

In [ ]:
df_forecast_test[['lq45_inv','pred_lq45_inv']].plot(figsize =(15,4))
df_forecast_valid[['lq45_inv','pred_lq45_inv']].plot(figsize =(15,4))
plt.show()

## Predict Real

In [ ]:
for i in range(1,11):
    i_to = -1 * i
    i_from = i_to - 1
    df_real = gf.load_pkl(pkl=r'..\data\processed\df_test_05.pkl')[i_from:i_to]
    df_real_pred = Predictor.forecasting(dataframe=df_real, name='pred_real')

    print(df_real_pred)